# Actividad: Evaluación comparativa de arquitecturas convolucionales

Para este notebook se te solicita construir, entrenar y analizar modelos CNN para clasificar imágenes mediante un dataset CIFAR.

**Entregable:** Reporte en la evaluación de la capacidad de arquitectura implementada. Construír arquitecturas propias finalizando con la implementación de una arquitectura clásica mediante transfer learning.


## Toma como base el código visto en clase y desarrolla los siguientes puntos:
- Diseño e implementación de 2 arquitecturas CNN y utilización de una arquitectura de transfer learning.

- Buen uso de data augmentation y regularización.

- Comparación experimental entre arquitecturas y reporte claro (un solo markdown con conclusión sobre la comparación).





In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Cargar y normalizar el dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

# Capas de Data Augmentation para robustecer el modelo
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
])


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


## Definiciones de modelos

In [ ]:
# --- Arquitectura 1: CNN Básica ---
model1 = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# --- Arquitectura 2: CNN Avanzada (Con Dropout y BatchNormalization) ---
model2 = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# --- Arquitectura 3: Transfer Learning (MobileNetV2) ---
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False
model_tl = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation='softmax')
])

/tmp/ipython-input-107506780.py:26: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## Entrenamiento de modelos.

In [ ]:
# Compilar y entrenar el Modelo 2 (el más completo)
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Entrenando Modelo 2...")
history2 = model2.fit(x_train, y_train_cat, epochs=5, validation_data=(x_test, y_test_cat))

# Compilar y entrenar el Modelo de Transfer Learning
model_tl.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("\nEntrenando Transfer Learning...")
history_tl = model_tl.fit(x_train, y_train_cat, epochs=5, validation_data=(x_test, y_test_cat))

## Estadística y gráficos

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history2.history['accuracy'], label='CNN Avanzada (Train)')
plt.plot(history2.history['val_accuracy'], label='CNN Avanzada (Val)')
plt.title('Precisión de los Modelos')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history2.history['loss'], label='CNN Avanzada (Loss)')
plt.plot(history_tl.history['loss'], label='Transfer Learning (Loss)')
plt.title('Pérdida (Loss) Comparativa')
plt.legend()

plt.show()

# Conclusiones.

Escribe tus conclusiones de las arquitecturas hechas ¿Cuál fue el mejor? ¿Por qué? ¿Qué mejoraría? ¿Cómo lo mejoraría?